In [ ]:
"""
Fix amounts for reservations with invoices but not amount or amount it eur
"""

In [1]:
import requests
import json
import os
from dictor import dictor
from threading import Thread
import queue


### step4
# Declare url and authentication with header secret FIXME protect this
###
headers = {'content-type': 'application/json', 'x-hasura-admin-secret': 'V3ntur4Pr0dH4sur4Tr4v3lC0re2o2o'}
url = 'https://core.venturatravel.org/v1/graphql'

def sendHasuraRequest(query, variables: dict={}):
    r = requests.post(url=url, json={'query': query, 'variables': variables}, headers=headers)
    return r.json()

In [2]:
def performRequest(query, ubjs, no_workers):
    class Worker(Thread):
        def __init__(self, request_queue):
            Thread.__init__(self)
            self.queue = request_queue
            self.results = []

        def run(self):
            while True:
                content = self.queue.get()
                if content == None:
                    break
                res = sendHasuraRequest(query, content)
                print("______________")
                print("Sending", content)
                print("Result", res)
                self.results.append(res)
                self.queue.task_done()

    # Create queue and add addresses
    q = queue.Queue()
    for ubj in ubjs:
        q.put(ubj)

    # Create workers and add tot the queue
    workers = []
    for _ in range(no_workers):
        worker = Worker(q)
        worker.start()
        workers.append(worker)
    # Workers keep working till they receive an empty string
    for _ in workers:
        q.put(None)
    # Join workers to wait till they finished
    for worker in workers:
        worker.join()

    # Combine results from all workers
    r = []
    for worker in workers:
        r.extend(worker.results)
    return r

In [10]:
queryReservations = """
query getReservations {
  reservation(where: {_and: [{amount: {_is_null: true}}, {invoices: {}}, { created_at: { _gt: "2018-01-01"}}, { status: {_neq: CANCELLED}}]}, order_by: {created_at: asc}) {
    id
    created_at
    status
    invoices(order_by: {version: desc_nulls_last}, limit: 1) {
      version
      total_amount_in_eur
      total_amount
    }
  }
}
"""
res = sendHasuraRequest(queryReservations)
reservations = res['data']['reservation']
print(len(reservations))

41


In [11]:
toUpdate = []
for r in reservations:
    obj = {
        'id': r['id'],
        'amount': r['invoices'][0]['total_amount'] or 0,
        'amount_in_eur': r['invoices'][0]['total_amount_in_eur'] or 0,
    }
    toUpdate.append(obj)

In [12]:
len(toUpdate)

41

In [13]:
queryUpdateReservation = """
mutation updateReservationAmount($id: Int!, $amount: numeric, $amount_in_eur: numeric) {
  update_reservation_by_pk(pk_columns: {id: $id}, _set: { amount: $amount,  amount_in_eur: $amount_in_eur}){
    id
  }
}
"""

In [14]:
results = performRequest(queryUpdateReservation, toUpdate, 6)
print("RESULTS")
print(results)

______________
Sending ______________
Sending {'id': 145771, 'amount': 0, 'amount_in_eur': 0}
Result {'data': {'update_reservation_by_pk': {'id': 145771}}}______________
Sending {'id': 143899, 'amount': 0, 'amount_in_eur': 0}
Result {'data': {'update_reservation_by_pk': {'id': 143899}}}

______________{'id': 143385, 'amount': 0, 'amount_in_eur': 0}
Sending 
Result{'id': 144340, 'amount': 0, 'amount_in_eur': 0}
Result {'data': {'update_reservation_by_pk': {'id': 144340}}}______________
Sending {'id': 145937, 'amount': 0, 'amount_in_eur': 0}
Result 
{'data': {'update_reservation_by_pk': {'id': 145937}}}
 {'data': {'update_reservation_by_pk': {'id': 143385}}}
______________
Sending {'id': 141700, 'amount': 0, 'amount_in_eur': 0}
Result {'data': {'update_reservation_by_pk': {'id': 141700}}}
____________________________
Sending {'id': 148110, 'amount': 0, 'amount_in_eur': 0}

Sending Result {'data': {'update_reservation_by_pk': {'id': 148110}}}
______________
Sending {'id': 148119, 'amount'